In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9001
2,Huelva,Confirmados PDIA 14 días,2268
3,Huelva,Tasa PDIA 14 días,"444,0589494129141"
4,Huelva,Confirmados PDIA 7 días,930
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,14
629,Municipio de Huelva sin especificar,Total Confirmados,118
630,Municipio de Huelva sin especificar,Curados,53


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9001.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2517.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 487 personas en los últimos 7 días 

Un positivo PCR cada 220 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9001.0,930.0,2268.0,510743.0,182.087664,444.058949,133.0
Huelva-Costa,4882.0,606.0,1353.0,288115.0,210.332680,469.604151,78.0
Condado-Campiña,2800.0,249.0,706.0,155057.0,160.586107,455.316432,38.0
Huelva (capital),2517.0,295.0,653.0,143663.0,205.341668,454.535963,30.0
Ayamonte,268.0,63.0,117.0,20946.0,300.773417,558.579204,15.0
Palos de la Frontera,180.0,30.0,83.0,11289.0,265.745416,735.228984,14.0
Lepe,773.0,108.0,226.0,27431.0,393.715140,823.885385,11.0
Moguer,258.0,44.0,121.0,22088.0,199.203187,547.808765,10.0
Sierra de Huelva-Andévalo Central,1202.0,61.0,193.0,67571.0,90.275414,285.625490,9.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,17.0,5.0,14.0,548.0,912.408759,2554.744526,0.0
Lucena del Puerto,58.0,13.0,31.0,3371.0,385.642243,919.608425,3.0
Palma del Condado (La),402.0,32.0,97.0,10761.0,297.370133,901.403215,3.0
Lepe,773.0,108.0,226.0,27431.0,393.715140,823.885385,11.0
Villarrasa,88.0,1.0,17.0,2176.0,45.955882,781.250000,0.0
Minas de Riotinto,61.0,11.0,30.0,3848.0,285.862786,779.625780,1.0
Palos de la Frontera,180.0,30.0,83.0,11289.0,265.745416,735.228984,14.0
Niebla,98.0,14.0,30.0,4117.0,340.053437,728.685936,4.0
Rociana del Condado,96.0,24.0,49.0,7855.0,305.537874,623.806493,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,88.0,1.0,17.0,2176.0,45.955882,781.250000,0.0,0.058824
Puebla de Guzmán,20.0,1.0,12.0,3073.0,32.541490,390.497885,0.0,0.083333
San Juan del Puerto,131.0,2.0,23.0,9300.0,21.505376,247.311828,0.0,0.086957
Valverde del Camino,257.0,6.0,38.0,12820.0,46.801872,296.411856,0.0,0.157895
Paterna del Campo,50.0,1.0,6.0,3509.0,28.498148,170.988886,0.0,0.166667
Villablanca,54.0,3.0,17.0,2848.0,105.337079,596.910112,0.0,0.176471
Campillo (El),52.0,2.0,11.0,2023.0,98.863075,543.746911,0.0,0.181818
Aracena,280.0,5.0,27.0,8107.0,61.675096,333.045516,2.0,0.185185
Jabugo,34.0,2.0,10.0,2250.0,88.888889,444.444444,0.0,0.200000
